In [0]:
"""
id,name,age,salary,address,gender
1,Manish,26,75000,INDIA,m
2,Nikita,23,100000,USA,f
3,Pritam,22,150000,INDIA,m
4,Prantosh,17,200000,JAPAN,m
5,Vikash,31,300000,USA,m
6,Rahul,55,300000,INDIA,m
7,Raju,67,540000,USA,m
8,Praveen,28,70000,JAPAN,m
9,Dev,32,150000,JAPAN,m
10,Sherin,16,25000,RUSSIA,f
11,Ragu,12,35000,INDIA,f
12,Sweta,43,200000,INDIA,f
13,Raushan,48,650000,USA,m
14,Mukesh,36,95000,RUSSIA,m
15,Prakash,52,750000,INDIA,m
"""

df = spark.read.format("csv")\
    .option("header", "true")\
    .option("inferschema", "true")\
    .load("/FileStore/tables/employee_write11_data.csv")

df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- salary: integer (nullable = true)
 |-- address: string (nullable = true)
 |-- gender: string (nullable = true)



In [0]:

df.write.format("csv")\
    .option("header", "true")\
    .option("mode", "overwrite")\
    .option("path", "/FileStore/tables/partition_by_address/")\
    .partitionBy("address")\
    .save()

In [0]:
dbutils.fs.ls("/FileStore/tables/partition_by_address/")

Out[16]: [FileInfo(path='dbfs:/FileStore/tables/partition_by_address/_SUCCESS', name='_SUCCESS', size=0, modificationTime=1743956505000),
 FileInfo(path='dbfs:/FileStore/tables/partition_by_address/address=INDIA/', name='address=INDIA/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/partition_by_address/address=JAPAN/', name='address=JAPAN/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/partition_by_address/address=RUSSIA/', name='address=RUSSIA/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/partition_by_address/address=USA/', name='address=USA/', size=0, modificationTime=0)]

In [0]:

df.write.format("csv")\
    .option("header", "true")\
    .option("mode", "overwrite")\
    .option("path", "/FileStore/tables/partition_by_address_gender/")\
    .partitionBy("address", "gender")\
    .save()

In [0]:
dbutils.fs.ls("/FileStore/tables/partition_by_address_gender/address=INDIA")

Out[19]: [FileInfo(path='dbfs:/FileStore/tables/partition_by_address_gender/address=INDIA/gender=f/', name='gender=f/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/partition_by_address_gender/address=INDIA/gender=m/', name='gender=m/', size=0, modificationTime=0)]

In [0]:
# Do trial and error before finalizing bucketBy number
# bucketBy does not work on file system, need to store data as a table (Hive metastore will store table metadata in this case)

df.write.format("csv")\
    .option("header", "true")\
    .option("mode", "overwrite")\
    .option("path", "/FileStore/tables/bucket_by_id/")\
    .bucketBy(3, "id")\
    .save()

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-1226002994629760>:3
      1 # Do trial and error before finalizing bucketBy number
----> 3 df.write.format("csv")\
      4     .option("header", "true")\
      5     .option("mode", "overwrite")\
      6     .option("path", "/FileStore/tables/bucket_by_id/")\
      7     .bucketBy(3, "id")\
      8     .save()

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:1395, in DataFrameWriter.save(self, path, format, mode, partitionBy, **options)
   1393     

In [0]:

df.write.format("csv")\
    .option("header", "true")\
    .option("mode", "overwrite")\
    .option("path", "/FileStore/tables/bucket1_by_id/")\
    .bucketBy(3, "id")\
    .saveAsTable("bucket1_by_id")

"""
Advantages:
Bucket Pruning which is similar to partition pruning 
"""

In [0]:
dbutils.fs.ls("/FileStore/tables/bucket1_by_id")

Out[27]: [FileInfo(path='dbfs:/FileStore/tables/bucket1_by_id/_SUCCESS', name='_SUCCESS', size=0, modificationTime=1743957174000),
 FileInfo(path='dbfs:/FileStore/tables/bucket1_by_id/_committed_8328350804713090700', name='_committed_8328350804713090700', size=306, modificationTime=1743957174000),
 FileInfo(path='dbfs:/FileStore/tables/bucket1_by_id/_started_8328350804713090700', name='_started_8328350804713090700', size=0, modificationTime=1743957174000),
 FileInfo(path='dbfs:/FileStore/tables/bucket1_by_id/part-00000-tid-8328350804713090700-7f097269-c6b0-4f07-a096-a6e7541d6cdb-49-1_00000.c000.csv', name='part-00000-tid-8328350804713090700-7f097269-c6b0-4f07-a096-a6e7541d6cdb-49-1_00000.c000.csv', size=270, modificationTime=1743957174000),
 FileInfo(path='dbfs:/FileStore/tables/bucket1_by_id/part-00000-tid-8328350804713090700-7f097269-c6b0-4f07-a096-a6e7541d6cdb-49-2_00001.c000.csv', name='part-00000-tid-8328350804713090700-7f097269-c6b0-4f07-a096-a6e7541d6cdb-49-2_00001.c000.csv', si